Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

import camb
from camb import model, initialpower

In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'

In [4]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [5]:
#Fiducial cosmological parameters
c=3e5
hubble=0.675
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9

nss = 0.9655
alphass = 0.0
betass = 0.0

gamma=0.545

kpivot = 0.05 

In [6]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass, nrun=alphass, nrunrun=betass, pivot_scalar=kpivot)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [7]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [8]:
#Get matter power spectrum at z=0: P(k,z=0)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-5, maxkh=1.0, npoints = 800)

In [9]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [10]:
#Derivatives

def dlogPd_dns(kk):
    return np.log(kk/kpivot)

def dlogPd_dalphas(kk):
    return (1./2)*pow(np.log(kk/kpivot),2)

def dlogPd_dbetas(kk):
    return (1./6)*pow(np.log(kk/kpivot),3)

In [11]:
#Redshift bins

zlist = np.arange(0.8,2.5,0.1)
ztest = zlist[10]
Nzbins = len(zlist)

#print zlist
print "ztest =", ztest
print "Number of redshift bins =", Nzbins

ztest = 1.8
Number of redshift bins = 17


In [12]:
midfreq = 1420.4e6 #Hz

#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00
#Define the comoving distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

def ynu(zc):
    return c*pow(1+zc,2)/(midfreq*Hz(zc))

print ztest, rcom(ztest), DA(ztest)

1.8 5022.5527092 1793.76882471


In [13]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.948607572137


In [14]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest)

0.446367370944


In [15]:
#fiducial bHI from Bull et al 2015
def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#fiducial OmHI Mario's fit
def OmHI(zc):
    return 0.00048+0.00039*zc-0.000065*pow(zc,2)

print bHI(ztest), OmHI(ztest)

1.156 0.0009714


In [16]:
#Construct P(k,μ,z) 
def beta(zc):
    return fg(zc)/bHI(zc)

def Pkz(kk,mu,zc):    
    return pow(1+beta(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [17]:
#mean brightness temperature [mK] Mario's fit
def Tb(zc):
    return 0.0559+0.2324*zc-0.024*pow(zc,2)

print ztest, Tb(ztest)

1.8 0.39646


In [18]:
#Construct P_HI(k,μ,z), Tb assumed known
def PHI(kk,mu,zc):
    return pow(bHI(zc),2)*Pkz(kk,mu,zc)

In [19]:
def lam(zc):
    return 21*(1+zc)

def thetab(zc):
    return lam(zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

In [20]:
#Interferometer noise specs

Tsys = 50.0*1e3 #mK
Nd = 1024 #number of elements
Ddish = 6.0*100 #cm 
Dmax = 250.0*100 #max baseline in cm

#Define (instantaneous) FOV separately
def FOV(zc): 
    return pow(lam(zc)/Ddish,2)

Area = 15000 #deg^2

fsky=Area/41253
print "fsky =", fsky

omegatot = Area*pow(pi/180,2)
ttotal = 10000*60*60 #sec

Dzbin = 0.1

#filling factor check (has to be < 1)
ff = Nd*(Ddish/Dmax)**2
print "fcover =", ff

#noise PS
def Pnoise(kk,zc,Tsys):
    Ae = pi*(Ddish/2)**2
    nu = (Nd*(Nd-1))*lam(zc)**2/(2*pi*(Dmax**2-Ddish**2)) #full expression for nu
    np = 2
    #note division with Tb^2
    return (1/pow(Tb(zc),2))*(lam(zc)**4*rcom(zc)**2*ynu(zc)*Tsys**2*omegatot)/(np*Ae**2*FOV(zc)*nu*ttotal) 

fsky = 0.363609919279
fcover = 0.589824


In [21]:
def kmin(zc): 
    return 2*pi*Ddish/(rcom(zc)*lam(zc))

def kmax_int(zc): #instrumental cutoff
    return 2*pi*Dmax/(rcom(zc)*lam(zc))

def kmax(zc): #non-linear cutoff (Smith et al 2003)
    kNL = 0.1
    #kNL = 0.14
    return kNL*pow(1+zc,2/(2+nss))

for zi in range(0,Nzbins):
    zc = zlist[zi]
    print zc, kmin(zc), kmax(zc), kmax_int(zc)

0.8 0.0346395852884 0.14864884108 1.44331605368
0.9 0.0299736312171 0.154169231521 1.24890130071
1.0 0.026326219257 0.159595782335 1.09692580237
1.1 0.0234068859676 0.164934667994 0.975286915319
1.2 0.0210243041126 0.170191383714 0.876012671357
1.3 0.0190476660664 0.17537084751 0.793652752766
1.4 0.0173847314716 0.180477483171 0.724363811315
1.5 0.0159687332006 0.185515288356 0.665363883359
1.6 0.0147502467758 0.190487890971 0.614593615656
1.7 0.0136919664849 0.195398596213 0.570498603537
1.8 0.0127652489679 0.200250426112 0.531885373665
1.9 0.0119477669927 0.205046153 0.497823624698
2.0 0.0112218806076 0.209788328003 0.467578358652
2.1 0.0105734835009 0.214479305448 0.440561812539
2.2 0.00999117125922 0.219121263881 0.416298802467
2.3 0.00946563198218 0.223716224248 0.394401332591
2.4 0.00898919328586 0.228266065716 0.374549720244


In [22]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

#effective volume going in the Fisher matrix
def Veff(kk,mu,zc):
    return Vsur(zc)*(PHI(kk,mu,zc)/(PHI(kk,mu,zc)+Pnoise(kk,zc,Tsys)))**2

print "%.4g" % Vsur(ztest)

1.872e+10


In [23]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [24]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [25]:
mugrid = np.linspace(-1., 1., 513) 

In [26]:
#Fisher matrix parameters 
params = ["0:ns","1:alphas","2:betas"]

In [27]:
#%%time
#   Fisher matrix   # 

Npar = 2 #Npar = 3
#create array of zeros
s = Npar

Fishermat = np.zeros([s,s])

for zi in range(0,Nzbins):
    zc = zlist[zi]
    if kmax_int(zc) < kmax(zc): kmaxx = kmax_int(zc)
    else: kmaxx = kmax(zc)    
    kgrid = np.linspace(kmin(zc), kmaxx, 513)
    if kpivot < kmin(zc) or kpivot > kmaxx: raise ValueError("kpivot out of range")
    K, MU = np.meshgrid(kgrid, mugrid)
    for i in range(0,Npar):  
        def deriv_i(kk,mu,zc):
            if i==0:  return dlogPd_dns(kk)
            elif i==1:  return dlogPd_dalphas(kk)
            elif i==2:  return dlogPd_dbetas(kk)
            else: print "out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlogPd_dns(kk)
                    elif j==1:  return dlogPd_dalphas(kk)
                    elif j==2:  return dlogPd_dbetas(kk)
                    else: print "index out of range" 
                Fishermat[i][j] += integrate2D(dF(K,MU),kgrid,mugrid)
            else: Fishermat[i,j] = Fishermat[j,i]

In [28]:
print Fishermat

[[ 3840051.35614167  2139024.24106506]
 [ 2139024.24106506  1272443.28746716]]


In [29]:
print linalg.inv(Fishermat)

[[  4.09378916e-06  -6.88181103e-06]
 [ -6.88181103e-06   1.23544686e-05]]


In [30]:
#print marginalised uncertainties on (ns,alphas,betas)
for i in range(0,Npar):
    print np.sqrt(linalg.inv(Fishermat)[i,i])

0.00202331143477
0.00351489240046
